In [ ]:
import numpy as np

# Define page size, page table size, and physical memory size
PAGE_SIZE = 4096               # 4KB
BYTES_PER_LINE = 8             # 8B
PHYSICAL_MEMORY_SIZE = 2**32   # 4GB
TABLE_SIZE = 2**30             # 1GB

PAGE_SIZE_2 = 2**21            # 2MB

FRAME_LEN = PAGE_SIZE // BYTES_PER_LINE
FRAME_LEN_2 = PAGE_SIZE_2 // BYTES_PER_LINE

PAGE_MASK = ((1 << 9) - 1) << 12
PD_PT_MASK = ((1 << 18) - 1) << 21
PML4_MASK = ((1 << 9) - 1) << 39

kernel_fp = [0, 0]
count = [0, 0]

# Define physical memory
physical_memory = np.full((PHYSICAL_MEMORY_SIZE // PAGE_SIZE), None)

# Define page tables
pml4_table = np.full((FRAME_LEN), None)
pd_pt_table = np.full((5, FRAME_LEN_2), None)
pt_table  = np.full((TABLE_SIZE // PAGE_SIZE, FRAME_LEN), None)

# Define FIFO replacement variables 
entry_time = [0] 
fifo = {}

def get_physical_address(virtual_address):

    # Extract the page offset
    page_offset = virtual_address & ((1 << 9) - 1)

    # Extract the page table indices
    pml4_index = (virtual_address & PML4_MASK) >> 39
    pd_pt_index = (virtual_address & PD_PT_MASK) >> 21
    pt_index = (virtual_address & PAGE_MASK) >> 12

    # Check if the PML4 entry is present
    if pml4_table[pml4_index] is None:
        pml4_table[pml4_index] = kernel_fp[0] 
        kernel_fp[0] += 1

    # Check if the PD_PT entry is present
    pd_pt_base = pml4_table[pml4_index]
    if pd_pt_table[pd_pt_base][pd_pt_index] is None:
        pd_pt_table[pd_pt_base][pd_pt_index] = kernel_fp[1]
        kernel_fp[1] += 1

    # Check if the PT entry is present
    pt_base = pd_pt_table[pd_pt_base][pd_pt_index]
    if pt_table[pt_base][pt_index] is None:
        handle_page_fault(pt_base, pt_index, pd_pt_index)
        count[0] += 1
    else: 
        count[1] += 1
    

    # Calculate the physical address
    physical_frame_number = pt_table[pt_base][pt_index]
    physical_address = (physical_frame_number << 12) | page_offset

    return physical_address

# Define a function to handle a page fault
def handle_page_fault(pt_base, pt_index, pd_pt_index):
    # Find a free frame in physical memory
    for i, frame in enumerate(physical_memory):
        if frame is None:
            # Allocate the frame to the page
            physical_memory[i] = pd_pt_index * FRAME_LEN + pt_index         
            pt_table[pt_base][pt_index] = i
            fifo[(pt_base, pt_index )] = entry_time[0]
            entry_time[0] += 1
            return

    # If all frames are in use, perform page replacement
    # For simplicity, we'll use FIFO (first in, first out) replacement

    (base_min, index_min) = min(fifo, key=fifo.get)
  
    del fifo[(base_min, index_min)]

    # Allocate the frame to the new page
    allocated_frame = pt_table[base_min][index_min]
    physical_memory[allocated_frame] = pd_pt_index * FRAME_LEN + pt_index
    pt_table[pt_base][pt_index] = allocated_frame

    fifo[(pt_base, pt_index )] = entry_time[0]
    entry_time[0] += 1

    pt_table[base_min][index_min] = None


In [ ]:
import time
 

# Read and parse the trace file

start = time.time()
with open("pinatrace_btree_large.out") as file:
    i = 0
    for line in file:
        i += 1      
        if line != '#eof\n':
            virtual_address = line[18:32]
            virtual_address = int(virtual_address, 0)
            physical_address = get_physical_address(virtual_address)
        if(i % 100000000 == 0):
            print(i, time.time() - start)
end =  time.time()
print("Execution time in seconds: ",(end-start))

In [ ]:
# Tracks the number of kernel page table entries allocated
kernel_fp

In [ ]:
pml4_fill = 0
pml4_null = 0

for element in pml4_table:
    if element is not None:
        pml4_fill += 1
    else:
        pml4_null += 1

In [ ]:
# Tracks the number of pml4 entries that are not null
pml4_fill

In [ ]:
# Tracks the number of pml4 entries that are null
pml4_null

In [ ]:
pd_pt_fill = [0, 0, 0]
pd_pt_null = [0, 0, 0]

for i in range(kernel_fp[0]):
    for element in pd_pt_table[i]:
        if element is not None:
            pd_pt_fill[i] += 1
        else:
            pd_pt_null[i] += 1

In [ ]:
# Tracks the number of 2MB page table entries that are not null
pd_pt_fill

In [ ]:
# Tracks the number of 2MB page table entries that are null 
pd_pt_null

In [ ]:
pt_fill = 0
pt_null = 0

for i in range(kernel_fp[1]):
    for element in pt_table[i]:
        if element is not None:
            pt_fill += 1
        else:
            pt_null += 1

In [ ]:
# Tracks the number of 4KB PT page table entries that are not null
pt_fill

In [ ]:
# Tracks the number of 4KB PT page table entries that are null
pt_null

In [ ]:
# Page hit and miss count 
count